In [3]:
!pip install wandb
!pip install 'transformers[torch]' 
!pip install datasets 
!pip install evaluate

  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 873.1 kB/s eta 0:00:000:00:01


In [4]:
import wandb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from datasets import load_dataset
from transformers import AutoTokenizer, AlbertForSequenceClassification, AlbertConfig, Trainer, TrainingArguments, AutoModelForSequenceClassification
from transformers import default_data_collator

import evaluate

from torch.utils.data import DataLoader, TensorDataset

from tqdm import tqdm

from IPython.display import clear_output

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
def encode(examples):
    result = tokenizer(examples["text"], truncation=True, max_length=512, padding="max_length")
    return result

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

@torch.no_grad()
def test(model, loader, device, tqdm_desc):
    loss_log = []
    acc_log = []
    model.eval()
    loss_func = nn.CrossEntropyLoss()

    for input_ids, attention_mask, labels in tqdm(loader, desc=tqdm_desc):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        out = model(input_ids, attention_mask=attention_mask)
        loss = loss_func(out.logits, labels)

        loss_log.append(loss.item())

        pred = torch.argmax(out.logits, dim=1)
        acc_log.append((pred == labels).detach().cpu().numpy().sum() / len(pred))

    return loss_log, acc_log


def train_epoch(model, optimizer, train_loader, device, tqdm_desc):
    loss_log = []
    acc_log = []
    model.train()
    loss_func = nn.CrossEntropyLoss()

    for input_ids, attention_mask, labels in tqdm(train_loader, desc=tqdm_desc):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        out = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = loss_func(out.logits, labels)
        loss.backward()
        optimizer.step()

        loss_log.append(loss.item())

        pred = torch.argmax(out.logits, dim=1)
        acc_log.append((pred == labels).detach().cpu().numpy().sum() / len(pred))

    return loss_log, acc_log


def train(model, optimizer, n_epochs, train_loader, val_loader, batch_size, scheduler=None):
    train_loss_log, train_acc_log, val_loss_log, val_acc_log = [], [], [], []
    train_len_epoch_loss = []
    train_len_epoch_acc = []
    
    run = wandb.init(project='html classificator', reinit=True)
    
    wandb.watch(model, nn.CrossEntropyLoss(), log="all", log_freq=1)

    count = len(train_loader) // batch_size
    for epoch in range(n_epochs):
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        train_loss, train_acc = train_epoch(model, optimizer, train_loader, device, tqdm_desc=f'Training {epoch}/{n_epochs}')
        val_loss, val_acc = test(model, val_loader, device, tqdm_desc=f'Validating {epoch}/{n_epochs}')

        train_loss_log.extend(train_loss)
        train_acc_log.extend(train_acc)

        val_loss_log.extend(val_loss)
        val_acc_log.extend(val_acc)

        # wandb
        wandb.log({"Accuracy": wandb.plot.line_series(
                xs=list(range((epoch + 1) * count)),
                ys=[train_acc_log, val_acc_log],
                keys=["train", "val"],
                title="Accuracy",
                xname="iter")})
        
        wandb.log({"Loss": wandb.plot.line_series(
                xs=list(range((epoch + 1) * count)),
                ys=[train_loss_log, val_loss_log],
                keys=["train", "val"],
                title="Loss",
                xname="iter")})

#         clear_output()
        print(f"Epoch {epoch + 1}")
        print(f" train loss: {np.mean(train_loss)}, train acc: {np.mean(train_acc)}")
        print(f" val loss: {np.mean(val_loss)}, val acc: {np.mean(val_acc)}\n")

        if scheduler is not None:
            scheduler.step()

    run.finish()
    return train_loss_log, train_acc_log, val_loss_log, val_acc_log

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [7]:
# tokenizer = AutoTokenizer.from_pretrained("XSY/albert-base-v2-imdb-calssification")
# model = AlbertForSequenceClassification(AlbertConfig()).cuda()

metric = evaluate.load("accuracy")

tokenizer = AutoTokenizer.from_pretrained("XSY/albert-base-v2-imdb-calssification")
model = AutoModelForSequenceClassification.from_pretrained("XSY/albert-base-v2-imdb-calssification").to(device)

tokenizer_config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/245 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/889 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

In [8]:
dataset = load_dataset("imdb")
tokenized_datasets = dataset.map(encode, batched=True, remove_columns="text")

train_dataset = TensorDataset(torch.tensor(tokenized_datasets['train']['input_ids']), torch.tensor(tokenized_datasets['train']['attention_mask']), torch.tensor(tokenized_datasets['train']['label']))
test_dataset = TensorDataset(torch.tensor(tokenized_datasets['test']['input_ids']), torch.tensor(tokenized_datasets['test']['attention_mask']), torch.tensor(tokenized_datasets['test']['label']))

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [8]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [9]:
wandb.login(key="eba16103be2afd0b5c96243771d60f5d7e562f68")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [28]:
# model = model.to(device)
val_loss, val_acc = test(model, val_loader, device, tqdm_desc='Test')
print(f'Без обучения на датасете (чисто модель с параметрами) loss = {np.mean(val_loss)}, accuracy = {np.mean(val_acc)}')

Test: 100%|██████████| 1563/1563 [15:43<00:00,  1.66it/s]

Без обучения на датасете (чисто модель с параметрами) loss = 0.1985105358434201, accuracy = 0.9360204734484965


In [14]:
# model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=2e-5, weight_decay=0.01)
train_loss_log, train_acc_log, val_loss_log, val_acc_log = train(model, optimizer, 3, train_loader, val_loader, batch_size)

Validating 0/3: 100%|██████████| 1563/1563 [19:23<00:00,  1.34it/s]


Epoch 1
 train loss: 0.16785732328043257, train acc: 0.9389795265515035
 val loss: 0.1752058006864773, val acc: 0.9335812539987204



Validating 1/3: 100%|██████████| 1563/1563 [19:23<00:00,  1.34it/s]


Epoch 2
 train loss: 0.13657005596608018, train acc: 0.9504958413307741
 val loss: 0.21571108164794275, val acc: 0.9181062060140754



Validating 2/3: 100%|██████████| 1563/1563 [19:22<00:00,  1.34it/s]


Epoch 3
 train loss: 0.11670218710862076, train acc: 0.9582933461292387
 val loss: 0.2059667423575752, val acc: 0.9221049264235445



In [12]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [11]:
optimizer = optim.Adam(model.parameters(), lr=2e-5, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=3, eta_min=3e-10)
train_loss_log, train_acc_log, val_loss_log, val_acc_log = train(model, optimizer, 3, train_loader, val_loader, batch_size, scheduler)

wandb: Currently logged in as: rodion-chernomordin. Use `wandb login --relogin` to force relogin


Validating 0/3: 100%|██████████| 1563/1563 [22:21<00:00,  1.17it/s]


Epoch 1
 train loss: 0.1664096171725172, train acc: 0.9387396033269354
 val loss: 0.18684211791530536, val acc: 0.9258237364043506



Validating 1/3: 100%|██████████| 1563/1563 [20:55<00:00,  1.24it/s]


Epoch 2
 train loss: 0.11898303203617228, train acc: 0.9587731925783749
 val loss: 0.215617141500026, val acc: 0.9220649392194498



Validating 2/3: 100%|██████████| 1563/1563 [22:04<00:00,  1.18it/s]


Epoch 3
 train loss: 0.05993715412082044, train acc: 0.9812859884836852
 val loss: 0.21360904601218164, val acc: 0.93170185540627



In [8]:
model_name = "albert"
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

wandb.login(key="eba16103be2afd0b5c96243771d60f5d7e562f68")

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.158600,0.177705,0.936000
2,0.087800,0.211011,0.937120
3,0.053800,0.286730,0.939360


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=2346, training_loss=0.09775648580487732, metrics={'train_runtime': 5395.4903, 'train_samples_per_second': 13.9, 'train_steps_per_second': 0.435, 'total_flos': 1792357632000000.0, 'train_loss': 0.09775648580487732, 'epoch': 3.0})